In [39]:
# import the libraries
import requests
from bs4 import BeautifulSoup

## Scraping all pages

In [40]:
# we'll be using the cars and conversations forum
url = 'https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations'

In [41]:
# number of forum pages
num_pages = 2572

In [68]:
# store the comments in a string
all_comments = ""

# go through each page
for page_number in range(1, num_pages+1):
    # first page string of url isn't anything special
    try:
        if(page_number == 1):
            new_url = url
        else:
            # end url format is /p220 etc.
            new_url = url + "/p" + str(page_number)
        print(new_url)
        # collect the url with requests library
        page = requests.get(new_url)
    
        # get the html of the page in string form
        page_html = page.text
    
        # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
        soup = BeautifulSoup(page_html, 'html.parser')
    
        # inspect the elements from the webpage to know where stuff is
        # in this case the comments are under a <p> tag which are all under a <div class = "Message userContent"</div/ tag 
        # which are all under a <div class = "MessageList DataList Comments"> tag
    
        # Pull all text from the 'MessageList DataList Comments' divs
        messagelist = soup.find(class_ = "MessageList DataList Comments")
    
        # within the 'MessageList DataList Comments' divs pull all text from 'Message userContent' divs
        usermessages = messagelist.find_all(class_ =  "Message userContent")
    
        # now extract just the <p> tags from all comments! 
        for i in range(len(usermessages)):
            # a user comment might have multiple <p> tags
            [s.extract() for s in usermessages[i](class_ = 'UserQuote')]
            user_comments = usermessages[i].find_all('p')
            for j in range(len(user_comments)):
                # remove <a>, <img>, <br> tags embedded in <p>
                [s.extract() for s in user_comments[j]('a')]
                [s.extract() for s in user_comments[j]('img')]
                [s.extract() for s in user_comments[j]('br')]

            
                # add the users comments to the mega string
                all_comments += user_comments[j].prettify()
    
        # print some checks
        print('Page ' + str(page_number) + ' complete')
        all_comments += '/n -------------------------------------------------------------' + 'page ' + str(page_number)
    except:
        print('page ' + str(page_number) + ' failed')
        continue

https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p3
Page 3 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p4
Page 4 complete


In [69]:
print(all_comments)

<p>
 I'm 2nd post....yee haw!!!!!!!!
</p>
<p>
 Do I get a badge?
</p>
<p>
 Great, but will it be possible to refer to old thread?
</p>
<p>
 And one more thing how do I find and add a discussion to "My Bookmarks?"
</p>
<p>
 If you have posted in that discussion, click on "Participated" and all your past post topics are chronologically listed.
</p>
<p>
 If you are looking for a brand or model or a keyword, use the "Car forums" link at the top left of this page to get some ways of searching for models, brands, keywords, or poster names.
</p>
<p>
 if you want to bookmark this one, there is a little 5 point star next to the title up above.  just click that and it turns yellow.
</p>
<p>
 Thanks
 , it worked like a charm.
</p>
<p>
 Thanks imidazol97. I will try that later but stickguy's approach works for me.
</p>
<p>
 Thanks for that. While this new board loads faster it just seems a little cumbersome and not very user friendly.
</p>
<p>
 No, but it's almost like you're famous.  Just think, 

In [70]:
# let's do some string manipulation
# remove the newline characters, '<p>', '</p','said:'
parsedData = all_comments.replace('\n', '')
parsedData = parsedData.replace(r"<p>", "")
parsedData = parsedData.replace(r"</p>", "")
parsedData = parsedData.replace(r"</p>", "")
parsedData = parsedData.replace(r"said:", "")
parsedData = parsedData.replace(r":", "")
parsedData = parsedData.lower()

In [72]:
parsedData

' i\'m 2nd post....yee haw!!!!!!!! do i get a badge? great, but will it be possible to refer to old thread? and one more thing how do i find and add a discussion to "my bookmarks?" if you have posted in that discussion, click on "participated" and all your past post topics are chronologically listed. if you are looking for a brand or model or a keyword, use the "car forums" link at the top left of this page to get some ways of searching for models, brands, keywords, or poster names. if you want to bookmark this one, there is a little 5 point star next to the title up above.  just click that and it turns yellow. thanks , it worked like a charm. thanks imidazol97. i will try that later but stickguy\'s approach works for me. thanks for that. while this new board loads faster it just seems a little cumbersome and not very user friendly. no, but it\'s almost like you\'re famous.  just think, if this discussion goes on for years and years like the other did, visitors loading this discussion 

In [61]:
# throw this into a text file just in case
new_file = open('alltext.txt', 'w', encoding = 'utf-8')
new_file.write(parsedData)
new_file.close()

In [45]:
# convert model names to brands
import csv

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
file = open('models.csv', 'r')
csv_file = csv.reader(file)

# store the brand names
brands = []
for i in csv_file:
    parsedData = parsedData.replace(i[1], i[0])
    if(i[0] not in brands):
        brands.append(i[0])

print('Finished converting model names to brands')
# close the file
file.close()

Finished converting model names to brands


## Word Frequency

In [46]:
# grab word frequency using nltk library
import nltk
# stop words
from nltk.corpus import stopwords
nltk.download()
nltk.download('stopwords')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(parsedData)

In [48]:
# let's do a word count
from collections import Counter

# counter object
words = Counter()

# update counter with new words
words.update(tokens)

In [49]:
# check
words.most_common()[:100]

[('the', 352364),
 ('a', 246912),
 ('to', 216048),
 ('i', 213476),
 ('it', 198044),
 ('and', 174896),
 ('of', 105452),
 ('that', 102880),
 ('for', 90020),
 ('in', 84876),
 ('was', 82304),
 ('on', 59156),
 ('my', 54012),
 ('this', 51440),
 ('is', 48868),
 ('you', 46296),
 ('had', 46296),
 ('have', 43724),
 ('car', 43724),
 ('we', 43724),
 ('s', 41152),
 ('out', 41152),
 ('one', 38580),
 ('but', 36008),
 ('with', 36008),
 ('more', 33436),
 ('if', 33436),
 ('or', 33436),
 ('at', 33436),
 ('be', 30864),
 ('discussion', 30864),
 ('up', 30864),
 ('so', 30864),
 ('get', 28292),
 ('old', 28292),
 ('from', 28292),
 ('post', 25720),
 ('will', 25720),
 ('all', 25720),
 ('some', 25720),
 ('there', 25720),
 ('not', 25720),
 ('years', 25720),
 ('time', 25720),
 ('as', 25720),
 ('he', 25720),
 ('are', 23148),
 ('t', 23148),
 ('posts', 23148),
 ('about', 23148),
 ('m', 20576),
 ('just', 20576),
 ('like', 20576),
 ('almost', 20576),
 ('did', 20576),
 ('first', 20576),
 ('name', 20576),
 ('here', 20576)

So these are the most common words, but they are also the same in any english text. Therefore they are not very insightful. Hence, let's remove single characters, numbers, and common words.

In [50]:
# store the stop words
stopwords = stopwords.words('english')[:]

In [51]:
# lets do this again: remove stop words, single letter variable names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1)

In [52]:
# check
words.most_common()[:1000]

[('car', 43724),
 ('one', 38580),
 ('discussion', 30864),
 ('get', 28292),
 ('old', 28292),
 ('post', 25720),
 ('years', 25720),
 ('time', 25720),
 ('posts', 23148),
 ('like', 20576),
 ('almost', 20576),
 ('first', 20576),
 ('name', 20576),
 ('got', 20576),
 ('invoice', 20576),
 ('benz', 20576),
 ('money', 20576),
 ('little', 18004),
 ('new', 18004),
 ('sold', 18004),
 ('find', 15432),
 ('thanks', 15432),
 ('famous', 15432),
 ('100k', 15432),
 ('blah', 15432),
 ('story', 15432),
 ('went', 15432),
 ('bmw', 15432),
 ('missing', 15432),
 ('funds', 15432),
 ('buick', 15432),
 ('page', 12860),
 ('next', 12860),
 ('title', 12860),
 ('see', 12860),
 ('well', 12860),
 ('number', 12860),
 ('may', 12860),
 ('saw', 12860),
 ('least', 12860),
 ('hour', 12860),
 ('company', 12860),
 ('olds', 12860),
 ('good', 12860),
 ('click', 10288),
 ('looking', 10288),
 ('brands', 10288),
 ('want', 10288),
 ('later', 10288),
 ('board', 10288),
 ('wow', 10288),
 ('going', 10288),
 ('things', 10288),
 ('pretty', 

In [53]:
# now let's do this again but with only brand names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x in brands)

In [54]:
brands

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'car',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyndai kia',
 'hyundai',
 'hyundai,',
 'hyundai.',
 'infiniti',
 'kia',
 'kia,',
 'kia.',
 'lincoln',
 'mazda',
 'mercedes',
 'mercedes benz',
 'mercedes-benz',
 'mercury',
 'mitsubishi',
 'nissan',
 'nissan.',
 'pontiac',
 'problem',
 'saturn',
 'seat',
 'sedan',
 'subaru',
 'suzuki',
 'toyata',
 'toyota',
 'volkswagen',
 'volkwagen',
 'volvo']

In [55]:
# check
words.most_common()[:]

[('car', 43724),
 ('bmw', 15432),
 ('buick', 15432),
 ('mercedes', 10288),
 ('acura', 7716),
 ('chevrolet', 5144),
 ('volvo', 5144),
 ('ford', 2572)]